# Original Code for the Completion of Project 2

Group 10

Project Leader: Carson Crenshaw

Members: Sujith Panchumarthy, Zoe Averill

DS 4002

Last Updated: 10/23/2023

### Standard Imports

In [1]:
#Standard imports
import pandas as pd
import numpy as np
from google.colab import files

## MediaPipe

#### The following code is an adaption of [this](https://developers.google.com/mediapipe/solutions/vision/object_detector/python) guide and [this](https://colab.research.google.com/github/googlesamples/mediapipe/blob/main/examples/object_detection/python/object_detector.ipynb) example code for object detection using MediaPipe.

### Installs and Imports

In [2]:
#Install MediaPipe library
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 43.7 MB/s eta 0:00:00


In [3]:
#Image displaying imports
import cv2
from google.colab.patches import cv2_imshow

#Object detection imports
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [4]:
#Filepath for downloaded MediaPipe "EfficientDet-Lite2 (int8)" model
model_path = "/content/efficientdet_lite2.tflite"

###Detection Model: Images with Single Items

In [ ]:
#Image uploads
#Select all images from computer files
uploaded = files.upload()

for filename in uploaded:
  content = uploaded[filename]
  with open(filename, 'wb') as f:
    f.write(content)

#Record all image filenames
filenames = []
for filename in uploaded:
  print(filename)
  filenames.append(filename)

Saving apple_01.jpeg to apple_01.jpeg
Saving apple_02.jpeg to apple_02.jpeg
Saving apple_03.jpeg to apple_03.jpeg
Saving apple_04.jpeg to apple_04.jpeg
Saving apple_05.jpeg to apple_05.jpeg
Saving banana_01.jpeg to banana_01.jpeg
Saving banana_02.jpeg to banana_02.jpeg
Saving banana_03.jpeg to banana_03.jpeg
Saving banana_04.jpeg to banana_04.jpeg
Saving banana_05.jpeg to banana_05.jpeg
Saving broccoli_01.jpeg to broccoli_01.jpeg
Saving broccoli_02.jpeg to broccoli_02.jpeg
Saving broccoli_03.jpeg to broccoli_03.jpeg
Saving broccoli_04.jpeg to broccoli_04.jpeg
Saving broccoli_05.jpeg to broccoli_05.jpeg
Saving carrot_01.jpeg to carrot_01.jpeg
Saving carrot_02.jpeg to carrot_02.jpeg
Saving carrot_03.jpeg to carrot_03.jpeg
Saving carrot_04.jpeg to carrot_04.jpeg
Saving carrot_05.jpeg to carrot_05.jpeg
Saving orange_04.jpeg to orange_04.jpeg
Saving orange_05.jpeg to orange_05.jpeg
Saving orange_06.jpeg to orange_06.jpeg
Saving orange_07.jpeg to orange_07.jpeg
Saving orange_08.jpeg to orang

In [5]:
#Create an ObjectDetector object
base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.ObjectDetectorOptions(base_options=base_options,
                                       score_threshold=0.5)
detector = vision.ObjectDetector.create_from_options(options)

In [ ]:
#Dataframe to store initial, single item, detection results (NOT for basket images with multiple items)
accuracyDF = pd.DataFrame(columns=["File Name","Object","Detected?","Confidence","Accepted?"])

#Loop through uploaded files
for IMAGE_FILE in filenames:
  #Load the input image
  image = mp.Image.create_from_file(IMAGE_FILE)

  #Detect objects in the input image
  detection_result = detector.detect(image)

  #Get object name for accuracy calculations
  objectName = IMAGE_FILE.rsplit("_")[0]

  #Flag to check if object is detected
  flag = False
  #If objects are detected, loop through then
  if len(detection_result.detections) > 0:
    for detection in detection_result.detections:
      #If object is correctly detected, check confidence
      if objectName == detection.categories[0].category_name:
        #Mark that object was detected
        flag = True
        #If confidence is greater than or equal to 75%, object is considered detected
        if detection.categories[0].score >= 0.75:
          row = [IMAGE_FILE,objectName,"Yes",detection.categories[0].score,"Yes"]
          break
        #If confidence is less than than 75%, object is NOT considered detected
        else:
          row = [IMAGE_FILE,objectName,"Yes",detection.categories[0].score,"No"]
          break

  #If no objects were detected or none of the detected objects were the object, record that object wasn't detected
  if flag == False:
    row = [IMAGE_FILE,objectName,"No",np.nan,np.nan]

  #Append row to accuracy df
  accuracyDF.loc[len(accuracyDF)] = row

In [ ]:
#Displaying accuracy results
accuracyDF

,File Name,Object,Detected?,Confidence,Accepted?
0,apple_01.jpeg,apple,Yes,0.792969,Yes
1,apple_02.jpeg,apple,No,NaN,NaN
2,apple_03.jpeg,apple,Yes,0.808594,Yes
3,apple_04.jpeg,apple,No,NaN,NaN
4,apple_05.jpeg,apple,No,NaN,NaN
5,banana_01.jpeg,banana,Yes,0.808594,Yes
6,banana_02.jpeg,banana,Yes,0.761719,Yes
7,banana_03.jpeg,banana,Yes,0.777344,Yes
8,banana_04.jpeg,banana,Yes,0.777344,Yes
9,banana_05.jpeg,banana,Yes,0.808594,Yes


In [ ]:
#Exporting accuracy df
accuracyDF.to_csv('accuracy.csv', index=False)

In [6]:
#Function for outlining and displaying the detected objects in an image
MARGIN = 10  # pixels
ROW_SIZE = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
TEXT_COLOR = (255, 0, 0)  # red


def visualize(
    image,
    detection_result
) -> np.ndarray:
  """Draws bounding boxes on the input image and return it.
  Args:
    image: The input RGB image.
    detection_result: The list of all "Detection" entities to be visualize.
  Returns:
    Image with bounding boxes.
  """
  for detection in detection_result.detections:
    #Draw bounding_box
    bbox = detection.bounding_box
    start_point = bbox.origin_x, bbox.origin_y
    end_point = bbox.origin_x + bbox.width, bbox.origin_y + bbox.height
    cv2.rectangle(image, start_point, end_point, TEXT_COLOR, 3)

    #Draw label and score
    category = detection.categories[0]
    category_name = category.category_name
    probability = round(category.score, 2)
    result_text = category_name + ' (' + str(probability) + ')'
    text_location = (MARGIN + bbox.origin_x,
                     MARGIN + ROW_SIZE + bbox.origin_y)
    cv2.putText(image, result_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                FONT_SIZE, TEXT_COLOR, FONT_THICKNESS)

  return image

In [ ]:
#Load the input image
image = mp.Image.create_from_file("apple_01.jpeg")

#Detect objects in the input image
detection_result = detector.detect(image)

#Display the detection result
image_copy = np.copy(image.numpy_view())
annotated_image = visualize(image_copy, detection_result)
rgb_annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
cv2_imshow(rgb_annotated_image)

###Detection Model: Images with Multiple Items

#### NOTE: The below code cells for multiple items are the same as the cells above for single items. For ease of use and readability, only the image upload and detection result cells are included below. The previous cells for ObjectDetecor and visualization function creation must be run for the below detection result cell work. The only unnecessary cells for multiple item detection are the accuracyDF cells.

In [7]:
#Image uploads
#Select all images from computer files
uploaded = files.upload()

for filename in uploaded:
  content = uploaded[filename]
  with open(filename, 'wb') as f:
    f.write(content)

#Record all image filenames
filenames = []
for filename in uploaded:
  print(filename)
  filenames.append(filename)

Saving group_01.jpeg to group_01.jpeg
Saving group_02.jpeg to group_02.jpeg
Saving group_03.jpeg to group_03.jpeg
Saving group_04.jpeg to group_04.jpeg
Saving group_05.jpeg to group_05.jpeg
group_01.jpeg
group_02.jpeg
group_03.jpeg
group_04.jpeg
group_05.jpeg


In [ ]:
#Load the input image
image = mp.Image.create_from_file("group_01.jpeg")

#Detect objects in the input image
detection_result = detector.detect(image)

#Display the detection result
image_copy = np.copy(image.numpy_view())
annotated_image = visualize(image_copy, detection_result)
rgb_annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
cv2_imshow(rgb_annotated_image)